# Quantum Circuit Building Demo

This notebook demonstrates the quantum circuit building capabilities in quactuary, including:
- Basic circuit construction with CircuitBuilder
- Parameterized circuits for variational algorithms
- Common circuit templates
- Circuit optimization techniques

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.visualization import circuit_drawer

# Import quactuary quantum modules
from quactuary.quantum.circuits.builders import CircuitBuilder, ParameterizedCircuitBuilder
from quactuary.quantum.circuits.templates import (
    create_uniform_superposition,
    create_ghz_state,
    create_qft_circuit,
    create_amplitude_encoding_circuit,
    create_probability_distribution_loader,
    create_grover_oracle,
    create_diffusion_operator,
    create_variational_ansatz,
    create_hardware_efficient_ansatz
)
from quactuary.quantum.quantum_types import CircuitMetrics

## 1. Basic Circuit Building with CircuitBuilder

The `CircuitBuilder` class provides a fluent interface for constructing quantum circuits.

In [ ]:
# Create a simple quantum circuit using the builder pattern
builder = CircuitBuilder(num_qubits=4, name="simple_circuit")

# Build circuit with method chaining
circuit = (builder
    .add_hadamard_layer([0, 1])  # Apply H gates to qubits 0 and 1
    .add_entangling_layer("linear")  # Linear entanglement pattern
    .add_barrier()  # Visual separator
    .add_rotation_layer("y", [np.pi/4, np.pi/3, np.pi/2, np.pi])
    .add_measurement()  # Measure all qubits
    .build()
)

# Display the circuit
print("Simple Circuit:")
circuit.draw(output='mpl', style='iqp')

In [ ]:
# Demonstrate different entanglement patterns
patterns = ["linear", "circular", "all_to_all"]
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for i, pattern in enumerate(patterns):
    builder = CircuitBuilder(num_qubits=4)
    circuit = (builder
        .add_hadamard_layer()
        .add_entangling_layer(pattern)
        .build()
    )
    circuit.draw(output='mpl', ax=axes[i], style='iqp')
    axes[i].set_title(f'{pattern.title()} Entanglement')

plt.tight_layout()
plt.show()

## 2. Parameterized Circuits for Variational Algorithms

Parameterized circuits are essential for variational quantum algorithms like VQE and QAOA.

In [ ]:
# Create a parameterized circuit for variational algorithms
param_builder = ParameterizedCircuitBuilder(
    num_qubits=3, 
    num_params=9,  # 3 layers × 3 qubits
    param_prefix="θ"
)

# Build a variational circuit with 3 layers
for layer in range(3):
    param_builder = (param_builder
        .add_parameterized_rotation_layer('y')
        .add_entangling_layer('linear')
        .add_barrier()
    )

variational_circuit = param_builder.build()

print(f"Number of parameters: {variational_circuit.num_parameters}")
print(f"Parameters: {list(variational_circuit.parameters)}")
variational_circuit.draw(output='mpl', style='iqp')

In [ ]:
# Demonstrate parameter binding
# Generate random parameter values
param_values = np.random.uniform(0, 2*np.pi, variational_circuit.num_parameters)

# Bind parameters to create a concrete circuit
bound_circuit = variational_circuit.assign_parameters(param_values)

print("Circuit after parameter binding:")
bound_circuit.draw(output='mpl', style='iqp')

## 3. Circuit Templates

Pre-built circuit templates for common quantum computing patterns.

In [ ]:
# Create uniform superposition
uniform_circuit = create_uniform_superposition(num_qubits=5)
print("Uniform Superposition Circuit:")
uniform_circuit.draw(output='mpl')

In [ ]:
# Create GHZ state
ghz_circuit = create_ghz_state(num_qubits=4)
print("GHZ State Preparation Circuit:")
ghz_circuit.draw(output='mpl')

In [ ]:
# Create Quantum Fourier Transform circuit
qft_circuit = create_qft_circuit(num_qubits=4, insert_barriers=True)
print("Quantum Fourier Transform Circuit:")
qft_circuit.draw(output='mpl', fold=-1)

In [ ]:
# Create inverse QFT
iqft_circuit = create_qft_circuit(num_qubits=4, inverse=True, insert_barriers=True)
print("Inverse Quantum Fourier Transform Circuit:")
iqft_circuit.draw(output='mpl', fold=-1)

## 4. Amplitude Encoding for Classical Data

Encode classical data into quantum amplitudes for quantum algorithms.

In [ ]:
# Encode a simple probability distribution
probabilities = [0.1, 0.2, 0.3, 0.4]
prob_circuit = create_probability_distribution_loader(probabilities)

print(f"Encoding {len(probabilities)} probabilities into {prob_circuit.num_qubits} qubits")
print(f"Probabilities: {probabilities}")
prob_circuit.draw(output='mpl')

In [ ]:
# Encode arbitrary amplitudes
# Create a normalized vector representing a quantum state
amplitudes = np.array([1, 2, 3, 4, 5, 6, 7, 8], dtype=complex)
amplitudes = amplitudes / np.linalg.norm(amplitudes)  # Normalize

amp_circuit = create_amplitude_encoding_circuit(amplitudes)
print(f"Encoding {len(amplitudes)} amplitudes into {amp_circuit.num_qubits} qubits")
amp_circuit.draw(output='mpl')

## 5. Grover's Algorithm Components

Building blocks for Grover's quantum search algorithm.

In [ ]:
# Create Grover oracle for marked states
marked_states = [5, 10]  # Mark states |101⟩ and |1010⟩
oracle = create_grover_oracle(marked_states, num_qubits=4)

print(f"Oracle marking states: {marked_states}")
oracle.draw(output='mpl')

In [ ]:
# Create diffusion operator
diffusion = create_diffusion_operator(num_qubits=4)
print("Grover Diffusion Operator:")
diffusion.draw(output='mpl')

In [ ]:
# Complete Grover's algorithm circuit
n_qubits = 4
marked = [5, 10]
n_iterations = int(np.pi/4 * np.sqrt(2**n_qubits / len(marked)))

# Build complete Grover circuit
grover = QuantumCircuit(n_qubits)

# Initialize with uniform superposition
grover.h(range(n_qubits))

# Apply Grover iterations
oracle = create_grover_oracle(marked, n_qubits)
diffusion = create_diffusion_operator(n_qubits)

for i in range(n_iterations):
    grover.barrier()
    grover.compose(oracle, inplace=True)
    grover.compose(diffusion, inplace=True)

print(f"Grover's algorithm with {n_iterations} iterations:")
grover.draw(output='mpl', fold=-1)

## 6. Variational Ansatz Circuits

Different ansatz structures for variational quantum algorithms.

In [ ]:
# Create different variational ansätze
ansatz_configs = [
    {"entanglement": "linear", "rotation_blocks": ["ry", "rz"], "depth": 2},
    {"entanglement": "circular", "rotation_blocks": ["rx", "ry"], "depth": 3},
    {"entanglement": "full", "rotation_blocks": ["ry"], "depth": 2}
]

fig, axes = plt.subplots(3, 1, figsize=(12, 10))

for i, config in enumerate(ansatz_configs):
    ansatz = create_variational_ansatz(
        num_qubits=4,
        depth=config["depth"],
        entanglement=config["entanglement"],
        rotation_blocks=config["rotation_blocks"]
    )
    
    ansatz.draw(output='mpl', ax=axes[i], style='iqp')
    axes[i].set_title(
        f'Ansatz: {config["entanglement"]} entanglement, '
        f'{config["rotation_blocks"]} rotations, depth={config["depth"]}'
    )

plt.tight_layout()
plt.show()

In [ ]:
# Hardware-efficient ansatz
hw_ansatz = create_hardware_efficient_ansatz(num_qubits=5, depth=3)
print("Hardware-Efficient Ansatz:")
hw_ansatz.draw(output='mpl', fold=-1)

## 7. Circuit Metrics and Analysis

Analyze circuit properties using the CircuitMetrics class.

In [ ]:
# Analyze various circuits
circuits_to_analyze = [
    ("GHZ State", create_ghz_state(6)),
    ("QFT", create_qft_circuit(5)),
    ("Variational", create_variational_ansatz(4, depth=3)),
    ("Hardware Efficient", create_hardware_efficient_ansatz(4, depth=2)),
    ("Grover (2 iterations)", grover)
]

print("Circuit Metrics Analysis:")
print("-" * 80)
print(f"{'Circuit Name':<20} {'Qubits':<8} {'Depth':<8} {'Gates':<8} {'CNOTs':<8} {'1Q Gates':<10} {'2Q Gates':<10}")
print("-" * 80)

for name, circuit in circuits_to_analyze:
    metrics = CircuitMetrics.from_circuit(circuit)
    print(f"{name:<20} {metrics.num_qubits:<8} {metrics.depth:<8} "
          f"{metrics.gate_count:<8} {metrics.cnot_count:<8} "
          f"{metrics.single_qubit_gates:<10} {metrics.two_qubit_gates:<10}")

In [ ]:
# Visualize circuit depth vs gate count
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Collect metrics
names = []
depths = []
gate_counts = []
cnot_counts = []

for name, circuit in circuits_to_analyze:
    metrics = CircuitMetrics.from_circuit(circuit)
    names.append(name)
    depths.append(metrics.depth)
    gate_counts.append(metrics.gate_count)
    cnot_counts.append(metrics.cnot_count)

# Plot depth
ax1.bar(names, depths, color='skyblue')
ax1.set_ylabel('Circuit Depth')
ax1.set_title('Circuit Depth Comparison')
ax1.tick_params(axis='x', rotation=45)

# Plot gate counts
x = np.arange(len(names))
width = 0.35

ax2.bar(x - width/2, gate_counts, width, label='Total Gates', color='lightcoral')
ax2.bar(x + width/2, cnot_counts, width, label='CNOT Gates', color='lightgreen')
ax2.set_ylabel('Gate Count')
ax2.set_title('Gate Count Comparison')
ax2.set_xticks(x)
ax2.set_xticklabels(names)
ax2.tick_params(axis='x', rotation=45)
ax2.legend()

plt.tight_layout()
plt.show()

## 8. Custom Circuit Building Example

Combine builder patterns with custom operations for complex circuits.

In [ ]:
# Define a custom gate operation
def add_controlled_rotation_cascade(circuit, control_qubit, target_qubits, angles):
    """Add cascading controlled rotations."""
    for i, (target, angle) in enumerate(zip(target_qubits, angles)):
        circuit.cry(angle, control_qubit, target)
        if i < len(target_qubits) - 1:
            circuit.barrier([control_qubit, target])

# Build a complex circuit using custom operations
builder = CircuitBuilder(num_qubits=5, name="custom_circuit")

# Custom circuit construction
custom_circuit = (builder
    .add_hadamard_layer([0])  # Prepare control qubit
    .add_custom_gate(
        add_controlled_rotation_cascade,
        control_qubit=0,
        target_qubits=[1, 2, 3, 4],
        angles=[np.pi/4, np.pi/3, np.pi/2, np.pi]
    )
    .add_barrier()
    .add_entangling_layer("circular")
    .add_measurement([0, 1, 2])  # Partial measurement
    .build()
)

print("Custom Circuit with Controlled Rotation Cascade:")
custom_circuit.draw(output='mpl', style='iqp')

## Summary

This notebook demonstrated:

1. **CircuitBuilder**: Fluent interface for building quantum circuits
2. **ParameterizedCircuitBuilder**: Creating variational circuits with parameters
3. **Circuit Templates**: Pre-built circuits for common patterns (GHZ, QFT, Grover)
4. **Amplitude Encoding**: Loading classical data into quantum states
5. **Variational Ansätze**: Different structures for VQE/QAOA algorithms
6. **Circuit Metrics**: Analyzing circuit properties and complexity
7. **Custom Operations**: Extending builders with custom gate sequences

These building blocks form the foundation for implementing quantum algorithms in the quactuary framework.